In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 불러오기
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WhiteHat/malicious_phish_data_step3.csv')

In [5]:
# Feature는 Target('type') 제외한 나머지
x = df_train.drop(columns=['type'])

# Target은 'type'
y = df_train['type']

# 라이브러리 불러오기
from sklearn.model_selection import train_test_split

# 데이터 Set 나누기 (Train:Test = 8:2)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=77)

x_train

,len_url,len_sub_domain,len_root_domain,len_suffix,is_private,count_special_char,num_port,len_query,count_sub_domain,is_filter,ratio_alpha_numeric,count_url_dots,value_entropy_url,count_file_path
5707,0.132616,0.000000,0.258621,0.000000,0.0,0.083333,0.0,0.000000,0.0,1.0,0.000000,0.117647,0.446487,0.066667
1053,0.114695,0.025641,0.137931,0.000000,0.0,0.041667,0.0,0.000000,0.1,1.0,0.142857,0.000000,0.479596,0.133333
2817,0.132616,0.170940,0.258621,0.222222,0.0,0.000000,0.0,0.000000,0.1,0.0,0.050000,0.176471,0.477781,0.000000
4799,0.347670,0.000000,0.172414,0.055556,0.0,0.416667,0.0,0.293388,0.0,1.0,0.048193,0.117647,0.640000,0.066667
462,0.344086,0.000000,0.241379,0.055556,0.0,0.291667,0.0,0.000000,0.0,1.0,0.168675,0.117647,0.602103,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4832,0.286738,0.000000,0.172414,0.055556,0.0,0.500000,0.0,0.000000,0.0,1.0,0.090909,0.117647,0.490768,0.066667
9119,0.093190,0.000000,0.017241,0.055556,0.0,0.083333,0.0,0.000000,0.0,0.0,0.000000,0.058824,0.403974,0.200000
7832,0.071685,0.000000,0.258621,0.055556,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.058824,0.354886,0.133333
2283,0.060932,0.000000,0.189655,0.055556,0.0,0.041667,0.0,0.024793,0.0,0.0,0.500000,0.058824,0.434025,0.066667


In [6]:
# 라이브러리 임포트
import joblib

#model 저장
model_rf = joblib.load('/content/drive/MyDrive/Colab Notebooks/WhiteHat/model_rf_이정민 (2).pkl')
model_xgb = joblib.load('/content/drive/MyDrive/Colab Notebooks/WhiteHat/model_xgb_이정민.pkl')

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [10]:
# 기본 모델 선언하기
model_rf3 = RandomForestClassifier(random_state=77)

# 자동으로 탐색할 파라미터 설정
param_grid = [ {'n_estimators':[100,200,300,400,500], 'max_depth':[10,20,30,40,50]}]
# GridSearchCV 선언하기
rf_grid = GridSearchCV(model_rf3,
                        param_grid, #param_grid에 있는 설정 값들로 지정
                        cv=3,#cv= cross validation / 테스트 케이스 분할 개념 / 넘어가도 됨
                        scoring='f1', # Classification일때  'accuracy','f1' ...
                                      # Regression 일때 'neg_mean_squared_error','r2'...
                        verbose=1           # Log 출력 Level 조정
                       )
# GridSearchCV 학습하기
rf_grid.fit(x_train, y_train)
print('최적의 파라미터 값 : ', rf_grid.best_params_)
print('최고의 점수 : ', rf_grid.best_score_)
model_rf_grid = rf_grid.best_estimator_
pred_rf_grid = model_rf_grid.predict(x_test)
print("F1 Score : ", f1_score(y_test, pred_rf_grid))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
최적의 파라미터 값 :  {'max_depth': 20, 'n_estimators': 500}
최고의 점수 :  0.9131979282390895
F1 Score :  0.9094922737306843


In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# 기본 모델 선언하기
model_xgb = XGBClassifier(random_state=77, use_label_encoder=False, eval_metric='logloss')

# 자동으로 탐색할 파라미터 설정
param_grid_xgb = [
    {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.05, 0.1, 0.2]
    }
]

# GridSearchCV 선언하기
xgb_grid = GridSearchCV(model_xgb,
                        param_grid_xgb,  # param_grid에 있는 설정 값들로 지정
                        cv=3,            # 3-fold 교차검증
                        scoring='f1',    # f1-score 기준으로 평가
                        verbose=1,       # Log 출력 Level 조정
                        n_jobs=-1        # 모든 코어 사용
                       )

# GridSearchCV 학습하기
xgb_grid.fit(x_train, y_train)

# 최적의 파라미터, 점수 출력
print('최적의 파라미터 값 : ', xgb_grid.best_params_)
print('최고의 점수 : ', xgb_grid.best_score_)

# 최적 모델로 테스트 데이터 예측
model_xgb_grid = xgb_grid.best_estimator_
pred_xgb_grid = model_xgb_grid.predict(x_test)

# F1 Score 출력
print("F1 Score : ", f1_score(y_test, pred_xgb_grid))


Fitting 3 folds for each of 80 candidates, totalling 240 fits
최적의 파라미터 값 :  {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 500}
최고의 점수 :  0.9181528182931338
F1 Score :  0.9195652173913044


In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# 기본 모델 선언하기
model_svm = SVC(random_state=77)

# 자동으로 탐색할 파라미터 설정
param_grid_svm = [
    {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    }
]

# GridSearchCV 선언하기
svm_grid = GridSearchCV(model_svm,
                        param_grid_svm,  # 탐색할 설정 값
                        cv=3,            # 3-fold 교차검증
                        scoring='f1',    # f1-score 기준으로 평가
                        verbose=1,       # 로그 출력 레벨
                        n_jobs=-1        # 모든 코어 사용
                       )

# GridSearchCV 학습하기
svm_grid.fit(x_train, y_train)

# 최적의 파라미터, 점수 출력
print('최적의 파라미터 값 : ', svm_grid.best_params_)
print('최고의 점수 : ', svm_grid.best_score_)

# 최적 모델로 테스트 데이터 예측
model_svm_grid = svm_grid.best_estimator_
pred_svm_grid = model_svm_grid.predict(x_test)

# F1 Score 출력
print("F1 Score : ", f1_score(y_test, pred_svm_grid))


Fitting 3 folds for each of 16 candidates, totalling 48 fits
최적의 파라미터 값 :  {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
최고의 점수 :  0.8724929035925623
F1 Score :  0.8565022421524664
